In [10]:
import random
import traceback
from heapq import heappop, heappush
from pathlib import Path
from textwrap import dedent
from typing import Callable, Dict, Iterable, List, Optional, Tuple, Type, Union

import matplotlib.pyplot as plt
import numpy as np
import numpy.typing as npt
from PIL import Image, ImageDraw
import pandas as pd
from tqdm import tqdm
epsilon = 1e-3
import torch
from torch.utils.data import DataLoader
from TransPath_model import TransPathModel, GridData, TransPathLit
from search_algorithms import Map, Node, SearchTreePQD, make_path, octile_distance, astar, astar_func, wastar

%matplotlib inline

In [4]:
dataset_path = './TransPath_data'
model_path = './weights/gold_cf_model'
mode = 'cf'
accelerator = "cuda"
devices = [6]
batch_size = 256

test_data = GridData(
        path=f'{dataset_path}/test',
        mode=mode
    )

dataloader = DataLoader(
        test_data, 
        batch_size=batch_size,
        shuffle=False, 
        # num_workers=multiprocessing.cpu_count(), freezes jupyter notebook
        pin_memory=True
    )

eval_model = TransPathModel().to(torch.device(f'{accelerator}:{devices[-1]}'))
eval_model.load_state_dict(torch.load(model_path, weights_only=True))
eval_model.eval()
predictions = []
for map_design, start, goal, gt_hmap in tqdm(dataloader):
    inputs = torch.cat([map_design, start + goal], dim=1) if mode in ('f', 'nastar') else torch.cat([map_design, goal], dim=1)
    inputs = inputs.to(torch.device(f'{accelerator}:{devices[-1]}'))
    
    with torch.no_grad():
        prediction = (eval_model(inputs) + 1) / 2

    predictions.append(prediction)

predictions = torch.cat(predictions).cpu().detach().numpy()

100%|███████████████████████████████████████████████████████████████████████| 250/250 [00:07<00:00, 34.64it/s]


In [5]:
np.save(f'{dataset_path}/test/predicted_cf.npy', predictions)

In [8]:
dataset_path = './TransPath_data'
model_path = './weights/gold_h_model'
mode = 'h'
accelerator = "cuda"
devices = [6]
batch_size = 256

test_data = GridData(
        path=f'{dataset_path}/test',
        mode=mode
    )

dataloader = DataLoader(
        test_data, 
        batch_size=batch_size,
        shuffle=False, 
        # num_workers=multiprocessing.cpu_count(), freezes jupyter notebook
        pin_memory=True
    )

eval_model = TransPathModel().to(torch.device(f'{accelerator}:{devices[-1]}'))
eval_model.load_state_dict(torch.load(model_path, weights_only=True))
eval_model.eval()
predictions = []
for map_design, start, goal, gt_hmap in tqdm(dataloader):
    inputs = torch.cat([map_design, start + goal], dim=1) if mode in ('f', 'nastar') else torch.cat([map_design, goal], dim=1)
    inputs = inputs.to(torch.device(f'{accelerator}:{devices[-1]}'))
    
    with torch.no_grad():
        prediction = (eval_model(inputs) + 1) / 2 * 64 * 64

    predictions.append(prediction)

predictions = torch.cat(predictions).cpu().detach().numpy()

100%|███████████████████████████████████████████████████████████████████████| 250/250 [00:06<00:00, 36.51it/s]


In [9]:
np.save(f'{dataset_path}/test/predicted_abs.npy', predictions)

In [12]:
dataset_path = './TransPath_data/test'

limit = 100

cells = np.load(f'{dataset_path}/maps.npy', mmap_mode='c')[:limit]
starts = np.load(f'{dataset_path}/starts.npy', mmap_mode='c')[:limit]
goals = np.load(f'{dataset_path}/goals.npy', mmap_mode='c')[:limit]

metrics = {'path_length' : [], 'expanded_nodes_num' : []}

for i in tqdm(range(cells.shape[0])):
    map = Map(cells[i, 0])
    start_i, start_j = np.where(starts[i, 0])
    start_i, start_j = start_i[0], start_j[0]
    goal_i, goal_j = np.where(goals[i, 0])
    goal_i, goal_j = goal_i[0], goal_j[0]
    data = astar_func(map, start_i, start_j, goal_i, goal_j, octile_distance, SearchTreePQD)
    assert data[0], draw(map, Node(start_i, start_j), Node(goal_i, goal_j))
    path, length = make_path(data[1])
    metrics['path_length'].append(length)
    metrics['expanded_nodes_num'].append(len(data[-1]))

100%|███████████████████████████████████████████████████████████████████████| 100/100 [00:02<00:00, 42.66it/s]


In [14]:
df = pd.DataFrame.from_dict(metrics)

In [15]:
df.to_csv('./vanilla_astar_octile_tie_break.csv', index=False)

In [16]:
dataset_path = './TransPath_data/test'

limit = 100

cells = np.load(f'{dataset_path}/maps.npy', mmap_mode='c')[:limit]
starts = np.load(f'{dataset_path}/starts.npy', mmap_mode='c')[:limit]
goals = np.load(f'{dataset_path}/goals.npy', mmap_mode='c')[:limit]
heuristics = np.load(f'{dataset_path}/predicted_abs.npy', mmap_mode='c')[:limit]

metrics = {'path_length' : [], 'expanded_nodes_num' : []}

for i in tqdm(range(cells.shape[0])):
    map = Map(cells[i, 0])
    start_i, start_j = np.where(starts[i, 0])
    start_i, start_j = start_i[0], start_j[0]
    goal_i, goal_j = np.where(goals[i, 0])
    goal_i, goal_j = goal_i[0], goal_j[0]
    data = astar(map, start_i, start_j, goal_i, goal_j, heuristics[i, 0], SearchTreePQD)
    assert data[0], draw(map, Node(start_i, start_j), Node(goal_i, goal_j))
    path, length = make_path(data[1])
    metrics['path_length'].append(length)
    metrics['expanded_nodes_num'].append(len(data[-1]))

100%|██████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 111.62it/s]


In [17]:
df = pd.DataFrame.from_dict(metrics)

In [18]:
df.to_csv('./vanilla_astar_hl.csv', index=False)

In [20]:
dataset_path = './TransPath_data/test'

limit = 100

cells = np.load(f'{dataset_path}/maps.npy', mmap_mode='c')[:limit]
starts = np.load(f'{dataset_path}/starts.npy', mmap_mode='c')[:limit]
goals = np.load(f'{dataset_path}/goals.npy', mmap_mode='c')[:limit]
heuristics = np.load(f'{dataset_path}/predicted_cf.npy', mmap_mode='c')[:limit]

metrics = {'path_length' : [], 'expanded_nodes_num' : []}

for i in tqdm(range(cells.shape[0])):
    map = Map(cells[i, 0])
    start_i, start_j = np.where(starts[i, 0])
    start_i, start_j = start_i[0], start_j[0]
    goal_i, goal_j = np.where(goals[i, 0])
    goal_i, goal_j = goal_i[0], goal_j[0]
    data = wastar(map, start_i, start_j, goal_i, goal_j, octile_distance, heuristics[i, 0], SearchTreePQD)
    assert data[0], draw(map, Node(start_i, start_j), Node(goal_i, goal_j))
    path, length = make_path(data[1])
    metrics['path_length'].append(length)
    metrics['expanded_nodes_num'].append(len(data[-1]))

100%|██████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 142.37it/s]


In [21]:
df = pd.DataFrame.from_dict(metrics)

In [22]:
df.to_csv('./vanilla_wastar_cf.csv', index=False)